In [1]:
!pip install openai "pyautogen[retrievechat]~=0.2.0b5" tiktoken cohere kaleido python-multipart chromadb --upgrade

In [15]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


In [26]:
import autogen

config_list = [
    {
        "model": "gpt-3.5-turbo",
        "api_key": "api-key"
    }
]
llm_config = {
    "seed": 42,
    "config_list": config_list,
    "temperature": 0
}

In [27]:
from chromadb.utils import embedding_functions
from langchain.text_splitter import RecursiveCharacterTextSplitter

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key = "api-key",
    model_name = "text-embedding-ada-002"
)

r_spliter = RecursiveCharacterTextSplitter(separators = ["\n", "\r", "\t"])

In [36]:
import chromadb
import autogen
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

assistant = autogen.AssistantAgent(
    name = "assistant",
    llm_config = llm_config,
    system_message = "You are a helpful assistant",
    is_termination_msg = lambda x: isinstance(x, dict) and "TERMINATION" == str(x.get("content", ""))[-9:].upper(),
)

retrieceuserproxy = RetrieveUserProxyAgent(
    name = "retreive_userproxy",
    system_message = "Assistant who has extra content retrieval power for solving difficult problems.",
    is_termination_msg = lambda x: isinstance(x, dict) and "TERMINATION" == str(x.get("content", ""))[-9:].upper(),
    human_input_mode = "NEVER",
    max_consecutive_auto_reply = 3,
    retrieve_config = {
        "task": "qa",
        "docs_path": "/content/cs224u-contextualreps-2023-handout.pdf",
        "custom_text_split_function": r_spliter.split_text,
        "embedding_function": openai_ef,
        "chunk_token_size": 1000,
        "model": "gpt-3.5-turbo-0613",
        # "collection_name": "groupchat",
        "get_or_create": True,

    },
    code_execution_config = False
)

In [37]:
assistant.reset()
retrieceuserproxy.initiate_chat(assistant, problem = "What is the role of postional encoding?")

Trying to create collection.


INFO:autogen.retrieve_utils:Found 23 chunks.


doc_ids:  [['doc_0', 'doc_3', 'doc_15', 'doc_18', 'doc_5', 'doc_2', 'doc_17', 'doc_11', 'doc_1', 'doc_16', 'doc_6', 'doc_21', 'doc_10', 'doc_14', 'doc_4', 'doc_8', 'doc_20', 'doc_19', 'doc_22', 'doc_9']]
Adding doc_id doc_0 to context.
retreive_userproxy (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What is the role of postional encoding?

Context is: Guiding ideas T ransformer Pos enc GPT BERT RoBERT a ELECTRA seq2seq Distillation Wrap-up
Contextual word representations
Christopher Potts
Stanford Linguistics
CS224u: Natural language understanding
1 / 81
Guiding ideas T ransformer Pos enc GPT BERT RoBERT a ELECTRA seq2seq Distillation Wrap-up
Guiding ideas
2 / 81Guiding ideas T ransformer Pos enc GPT BERT RoB